In [2]:
# load packages
import itertools
import pdb 
import time
from datetime import timedelta, datetime
from os.path import join
import os
from os import listdir
from os.path import isfile, join
import csv

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib inline

from mne.filter import filter_data

import scipy
from scipy.signal import welch, decimate, periodogram, find_peaks
from scipy import signal
from scipy import stats
from scipy.stats import pearsonr, mannwhitneyu, spearmanr
from scipy.ndimage.filters import uniform_filter1d
from statistics import mode
import math  

import sys


# Extracting bradykinesia features from downloaded Accelerometer data

The function requires already downloaded (Apple watch) accelerometry (acc) data, saved in csv-files, per day, per patient. Saved with filenames as in Notebook Data Download (e.g. 'RCS02_10Jun2020_userAcc.csv')
The function requires all acc-data files to be in one folder, with the patient-code as a name (e.g. RCS02).

The function will extract features per day, write csv files with all features per day, for every day in the defined time span in the function input.

In [2]:
path = os.path.join(os.path.dirname(os.getcwd())) # changed to main folder, instead of results
print(path)



/Users/roee/Starr_Lab_Folder/Data_Analysis/medStateDetection/results


## Load in Accelerometry data, bandpass Filter, and Extract Features




In [7]:
def load_filterWatchData(pt, y0,m0,d0,y1,m1,d1):
    
    '''
    Input: 
    - pt: patient as string (e.g. 'RCS02')
    - y0,m0,d0 : start date of desired timeperiod (year, month, date, e.g. 2020, 5, 1)
    - y1,m1,d1 : end date of desired timeperiod (year, month, date, e.g. 2020, 6, 1)
    
    Calculates features per day.
    Saves features as .csv
    
    Returns: one DF with raw AW data, and 1 DF with filtered AW data
    '''

    sr = 50 # sample ratio apple watch accelerometry, used in filter function below
    
    bandPassLow = 0 # lower cutoff of bandpass filter
    bandPassHigh = 3.5 # higher cutoff of bandpass filter
    
    filteredData = {} # empty dict to store filtered rcs data
    
    # define days in given timespan
    def datetime_range(start=None, end=None):
        span = end - start
        for i in range(span.days + 1):
            yield start + timedelta(days=i)
    # create list with datetime dates for every day in timespan
    datetimeDays = list(datetime_range(start=datetime(y0, m0, d0), end=datetime(y1, m1, d1)))
    
    # extract all file 'userAccel.csv'-filenames from specified patient-folder
    patient_dir_name = os.path.join(path,'data',pt)
    folderFiles= [s for s in listdir(patient_dir_name) if s[-15:] =='watch_accel.csv'] 
        
    for fileDay in datetimeDays: # loop over all days in requested timespan
        # define name of day-file
        day = fileDay.strftime("%d") # generate 2-digit day code
        month = fileDay.strftime("%d") # generate 2-digit month code
        year = fileDay.strftime("%Y") # generate 4-digit year code
        fileName = '%s_%s%s%s_watch_accel.csv' % (pt,year,month,day) # first pt is for specific pt-folder
        # check if acc-data file exist in folder, if not: skip day and continue with next
        if fileName in folderFiles:
            fileName = fileName # go on
        else:
            print('no file for %s' %fileName)
            continue # skips rest of itiration and takes next iteration
            
        # read csv file
        csv_full_path = os.path.join(path,pt,fileName)
        rawFile = pd.read_csv(csv_full_path , header=0) 
        
        ## DATA LOADING
        timeStamps = [] # create empty list for timestamps
        timeDelta = [0] # list for time difference per sample vs previous sample (0 for fist) (check for timestamp consistency)
        for row in np.arange(len(rawFile['time'])): # loop over every sample
            timeStamps.append(datetime.fromtimestamp(rawFile['time'][row])) # add timestamp to list
            if row > 0: # add timediff to a list, except for first sample...
                timeDelta.append((timeStamps[row] - timeStamps[row-1]).total_seconds())
        # select only acc axes
        dat = rawFile[['x','y','z']].rename(columns={"x": "X", "y": "Y", "z": "Z"})
        # calculate raw SVM before filtering
        dat['SVM'] = np.sqrt(dat['X']**2 + dat['Y']**2 + dat['Z']**2 )
        dat.insert(loc=0, column='timeStamp', value=timeStamps) # add timestamps as first column
        # dat is now ready raw acc file
        
        ## DATA FILTERING
        dat = dat.sort_values(by=['timeStamp']).reset_index(drop=True) # sort by timestamp and reset indices
        # filter raw RCS acc with wrist-feature relevant bandwidths 
        # make new dataframe for filtered data, with same timestamps
        filtered = pd.DataFrame(data = dat['timeStamp'], columns = ['timeStamp'])
        
        for col in ['X' ,'Y', 'Z', 'SVM']: # loop over all acc-data columns to filter
            # bandpass filter excluding tremor frequencies > 4hz
            filteredCol = filter_data(np.array(dat[col]),sr,bandPassLow,bandPassHigh,method='iir',verbose='WARNING')
            # filtered data per column stored in dat, write dat to dataframe column in filtered data DF
            filtered[col] = filteredCol
        
        filteredData[day+month] = filtered
        
    
    return filteredData




## Feature Extraction 


In [4]:
## Bradykinesia faetures extracted from Github Mahadevan 2020
## Source: https://github.com/NikhilMahadevan/analyze-tremor-bradykinesia-PD

def histogram(signal_x):
    '''
    Calculate histogram of sensor signal.
    :param signal_x: 1-D numpy array of sensor signal
    :return: Histogram bin values, descriptor
    '''
    descriptor = np.zeros(3)

    ncell = np.ceil(np.sqrt(len(signal_x)))

    max_val = np.nanmax(signal_x.values)
    min_val = np.nanmin(signal_x.values)

    delta = (max_val - min_val) / (len(signal_x) - 1)

    descriptor[0] = min_val - delta / 2
    descriptor[1] = max_val + delta / 2
    descriptor[2] = ncell

    h = np.histogram(signal_x, ncell.astype(int), range=(min_val, max_val))

    return h[0], descriptor

def dominant_frequency(signal_df, sampling_rate, cutoff ):
    '''
    Calculate dominant frequency of sensor signals.
    :param signal_df: Pandas DataFrame housing desired sensor signals
    :param sampling_rate: sampling rate of sensor signal
    :param cutoff: desired cutoff for filter
    :param channels: channels of signal to measure dominant frequency
    :return: Pandas DataFrame of calculated dominant frequency for each signal channel
    '''
    dominant_freq_df = pd.DataFrame()

    signal_x = signal_df

    padfactor = 1
    dim = signal_x.shape
    nfft = 2 ** ((dim[0] * padfactor).bit_length())

    freq_hat = np.fft.fftfreq(nfft) * sampling_rate
    freq = freq_hat[0: int(nfft / 2)]

    idx1 = freq <= cutoff
    idx_cutoff = np.argwhere(idx1)
    freq = freq[idx_cutoff]

    sp_hat = np.fft.fft(signal_x, nfft)
    sp = sp_hat[0: int(nfft / 2)] * np.conjugate(sp_hat[0: int(nfft / 2)])
    sp = sp[idx_cutoff]
    sp_norm = sp / sum(sp)

    max_freq = freq[sp_norm.argmax()][0]
    max_freq_val = sp_norm.max().real

    idx2 = (freq > max_freq - 0.5) * (freq < max_freq + 0.5)
    idx_freq_range = np.where(idx2)[0]
    dom_freq_ratio = sp_norm[idx_freq_range].real.sum()

    # Calculate spectral flatness
    spectral_flatness = 10.0*np.log10(stats.mstats.gmean(sp_norm)/np.mean(sp_norm))

    # Estimate spectral entropy
    spectral_entropy_estimate = 0
    for isess in range(len(sp_norm)):
        if sp_norm[isess] != 0:
            logps = np.log2(sp_norm[isess])
        else:
            logps = 0
        spectral_entropy_estimate = spectral_entropy_estimate - logps * sp_norm[isess]

    spectral_entropy_estimate = spectral_entropy_estimate / np.log2(len(sp_norm))
    # spectral_entropy_estimate = (spectral_entropy_estimate - 0.5) / (1.5 - spectral_entropy_estimate)

    dominant_freq_df['_dom_freq_value'] = [max_freq]
    dominant_freq_df['_dom_freq_magnitude'] = [max_freq_val]
    dominant_freq_df['_dom_freq_ratio'] = [dom_freq_ratio]
    dominant_freq_df['_spectral_flatness'] = [spectral_flatness[0].real]
    dominant_freq_df['_spectral_entropy'] = [spectral_entropy_estimate[0].real]

    return dominant_freq_df

def signal_entropy(windowData):
    data_norm = windowData/np.std(windowData)
    h, d = histogram(data_norm)
    lowerbound = d[0]
    upperbound = d[1]
    ncell = int(d[2])

    estimate = 0
    sigma = 0
    count = 0

    for n in range(ncell):
        if h[n] != 0:
            logf = np.log(h[n])
        else:
            logf = 0
        count = count + h[n]
        estimate = estimate - h[n] * logf
        sigma = sigma + h[n] * logf ** 2

    nbias = -(float(ncell) - 1) / (2 * count)

    estimate = estimate / count
    estimate = estimate + np.log(count) + np.log((upperbound - lowerbound) / ncell) - nbias
    
    # Scale the entropy estimate to stretch the range
    estimate = np.exp(estimate ** 2) - np.exp(0) - 1


    
    return estimate







In [9]:

def extractFeatures(pt, filteredData, windowLen=60, sr=50):
    '''
    Input: 
    - filteredData = dictionary of filtered acc data, for every day a seperate dataframe
    filteredData is automatically result of first function.
    - windowLen = desired window length of features in seconds
    - sr = sample frequency of recorded accelerometry data, in Hz, AppleWatch accelerometry = 50 Hz.
    
    Writes feature dataframes per day to .csv
    
    Returns: One dictionary with feature dataframes.
    '''

    tDelta = sr*windowLen # time-delta is factor between filtered data sample rate and desired windowlength
    
    # Define all names of feature-labels which will be calculated over all axes
    totalFeatLabels = [] # one list for all feature names (SVM,X,Y and Z)
    
    for axis in ['SVM','X', 'Y', 'Z']:  # loop over all axes, calculate features per axis    
        # add list with features for every axis
        featureList = ['_maxAcc','_iqrAcc', '_90prcAcc','_medianAcc','_meanAcc',
                       '_stddev','_variance','_coefVar','_accRange',
                        '_lowPeaks','_highPeaks', '_time1gAcc','_accEntropy','_jerkRatio',
                       '_RMS', 
                       '_specPow_totalu4Hz', '_specPow_low','_specPow_mid', '_specPow_high',
         '_domFreq_magnitude', '_domFreq_ratio', 
                       '_spectral_flatness', '_spectral_entropy',] #'_domFreq_value', (left out, no variation)
        
        for feat in featureList:
            totalFeatLabels.append(axis+feat)
        # features only for XYZ
        if np.logical_or(axis == 'X', axis == 'Y'): # ratio RMS only relevant for x y and z
            totalFeatLabels.append(axis+'_ratioRMS')
        elif axis == 'Z':
            totalFeatLabels.append(axis+'_ratioRMS')    

        # features only once calculated, in SVM
        if axis == 'SVM':
            for l in ['_spectralVar','_spectralSmoothness1','_spectrallowPeaks',
                     '_spectralSmoothness2','_spectralhighPeaks']:
                totalFeatLabels.append(axis+l)
            totalFeatLabels.extend(['crossCor_XY','crossCor_XZ','crossCor_YZ'])
    
    ''' TotalFeatLabels is now a list with all feature labels of X,Y,Z,SVM.
    One dataframe per session will be calculated and afterwards merged into one total feature-dataframe.
    '''
    features = {} # empty dict to store feature-dataframes per day
    list_days = filteredData.keys() # define days to calculate features for
    for day in list_days: # loop over every day in filteredData
        # basis for new feature dataframe is timestamps of filteredData
        # Add timestamp of beginning of faeture-window to list for feature dataframe timestamps
        timeStamps = filteredData[day]['timeStamp'][::tDelta] # take every timestamp at beginning of a feature window
        features[day] = pd.DataFrame(data=timeStamps, columns=['timeStamp'])
        
        # create dict with empty lists for all feature-names 
        totalFeatureLists = {} # empty dict to store features in lists for this session
        for label in totalFeatLabels:
            totalFeatureLists[label] = []
        
        # CALCULATION OF FEATURES, PER AXIS
        for axis in ['SVM','X', 'Y', 'Z']:  # loop over all axes, calculate features per axis
            
            for windowStart in np.arange(0,len(filteredData[day][axis]),tDelta): # iterate over windows of 120 hz * 60 s       
                windowData = filteredData[day][axis][windowStart : windowStart+tDelta] # create windowdata per column and per window
                
                ## DISTRIBUTIVE AND DESCRIPTIVE FEATURES FROM TIME DOMAIN
                                
                # max acceleration (source: Griffiths 2012)
                maxAcc = np.max(np.abs(windowData)) 
                totalFeatureLists[axis+'_maxAcc'].append(maxAcc)
                
                # IQR of acc
                iqrAcc = scipy.stats.iqr(windowData)
                totalFeatureLists[axis+'_iqrAcc'].append(iqrAcc)
                
                # 90-th percentile acc, (Rispens 2015 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4296095/?report=classic )
                perc90 = np.percentile(np.abs(windowData), 90)
                totalFeatureLists[axis+'_90prcAcc'].append(perc90)
                
                # median of acc
                medianAcc = np.median(np.abs(windowData))
                totalFeatureLists[axis+'_medianAcc'].append(medianAcc)
                
                # mean of acc
                meanAcc = np.mean(np.abs(windowData))
                totalFeatureLists[axis+'_meanAcc'].append(meanAcc)
                
                # standard deviation
                stddev = np.std(np.abs(windowData))
                totalFeatureLists[axis+'_stddev'].append(stddev)
                
                # variance (var = mean(abs(x - x.mean())**2))
                var = np.var(np.abs(windowData))
                totalFeatureLists[axis+'_variance'].append(var)
                
                # Coefficient of variance (stddev / mean)
                coefVar = scipy.stats.variation(np.abs(windowData))
                totalFeatureLists[axis+'_coefVar'].append(coefVar)
                
                # range in signal value; from Mahadevan-Github 
                accRange = windowData.max(skipna=True) - windowData.min(skipna=True)
                totalFeatureLists[axis+'_accRange'].append( accRange )
                
                # number of acceleration peaks per axes
                # low threshold peaks: activity indication
                lowPeaks = len(find_peaks(np.abs(windowData),height=1, threshold=None, distance=600)[0]) # height = required value to be a peak, distance is horizontal distance to allow next peak
                totalFeatureLists[axis+'_lowPeaks'].append( lowPeaks )
                # high threshold peaks: amount of faster activity
                highPeaks = len(find_peaks(np.abs(windowData),height=3, threshold=None, distance=600)[0]) # height = required value to be a peak, distance is horizontal distance to allow next peak
                totalFeatureLists[axis+'_highPeaks'].append( highPeaks )
                
                # % time spent in above 1g acceleration
                time1gAcc = np.sum(np.abs(windowData) > 1)/len(windowData)
                totalFeatureLists[axis+'_time1gAcc'].append( time1gAcc )

                # entropy in accelerometry (source: Mahadevan-github)
                sigEntropy = signal_entropy(windowData)
                totalFeatureLists[axis+'_accEntropy'].append(sigEntropy)
                    
                ## jerk ratio/smoothness; rate of acc-changes (Hogan 2009) acc to Mahadevan, PM aimed for 3-sec windows
                ampl = np.max(np.abs(windowData))
                jerk = windowData.diff(1) * sr #(divided by 1 / sr => multiply with sr)
                jerkSqSum = np.sum(jerk ** 2)
                scale = 360 * ampl ** 2 / tDelta / sr
                meanSqJerk = jerkSqSum / sr / (tDelta / sr * 2)
                jerkRatio = meanSqJerk / scale
                totalFeatureLists[axis+'_jerkRatio'].append(jerkRatio)                               
                
                # RMS according to classical definition
                meanSqAcc = np.mean(np.square(windowData))
                rmsAcc = np.sqrt(meanSqAcc)
                totalFeatureLists[axis+'_RMS'].append(rmsAcc )
                
                # RMS ratio (RMS-axis / RMS-svm) (Sekine '13: https://www.ncbi.nlm.nih.gov/pubmed/24370075)
                # rms ratio in mediolateral direction is correlated with walking speed
                if  np.logical_or(axis == 'X' , axis == 'Y'):
                    svmRMS = np.sqrt(np.mean(np.square(filteredData[day]['SVM'][windowStart : windowStart+tDelta])))
                    ratioRMS = rmsAcc / svmRMS
                    totalFeatureLists[axis+'_ratioRMS'].append(ratioRMS)
                elif axis ==  'Z':
                    svmRMS = np.sqrt(np.mean(np.square(filteredData[day]['SVM'][windowStart : windowStart+tDelta])))
                    ratioRMS = rmsAcc / svmRMS
                    totalFeatureLists[axis+'_ratioRMS'].append(ratioRMS)  


                ## FEATURES FROM SPECTRAL DOMAIN                
               
                # Griffiths: MSP: not described: mean over whole 0.2-4.0? mean per which bin-width?? svm or axis?
                freq = np.fft.rfftfreq(len(windowData), d = 1/sr) # define freq's for rfft, resolution (bins/hz) is dependent on windowlength of data
                lowFreq = np.logical_and(freq < 3.5, freq > 0.0) # select freq's of interest, total = 0-30hz since sr=60
                rfft = np.fft.rfft(windowData) # real fast fourier transform (same as fft[freq > 0])
                psd = np.log(np.abs(rfft)**2) # log to normalize (rfft gives same barplot as periodogram and fft)
                ## ??? is PSD correct as squared value of magnitude, log for normalization?
                psdLow = np.sum(psd[lowFreq]) # sum of psd's between selected freq's
                totalFeatureLists[axis+'_specPow_totalu4Hz'].append(psdLow)

                ## Evers (preprint 2020) gait cadence in 0.7 - 1.4 hz and 1.4 - 2.8 hz
                freqGaitA = np.logical_and(freq < 1.4, freq > 0.7) # select freq's of interest, total = 0-30hz since sr=60
                psdGaitA = np.sum(psd[freqGaitA]) # sum of psd's between selected freq's
                totalFeatureLists[axis+'_specPow_low'].append(psdGaitA)

                freqGaitB = np.logical_and(freq < 2.8, freq > 1.4) # select freq's of interest, total = 0-30hz since sr=60
                psdGaitB = np.sum(psd[freqGaitB]) # sum of psd's between selected freq's
                totalFeatureLists[axis+'_specPow_mid'].append(psdGaitB)
                
                freqGaitC = np.logical_and(freq < 3.5, freq > 2.8) # select freq's of interest, total = 0-30hz since sr=60
                psdGaitC = np.sum(psd[freqGaitC]) # sum of psd's between selected freq's
                totalFeatureLists[axis+'_specPow_high'].append(psdGaitC)          
            
                # dom freq + ratio + spectral flatness and entropy (source: Mahadevan-github)
                domFreqValues = dominant_frequency(windowData, sr, 3) # 4 (3) = cutoff for spectrum too analyze
#                 totalFeatureLists[axis+'_domFreq_value'].append( float(domFreqValues['_dom_freq_value']))
                totalFeatureLists[axis+'_domFreq_magnitude'].append( float(domFreqValues['_dom_freq_magnitude']))
                totalFeatureLists[axis+'_domFreq_ratio'].append(float( domFreqValues['_dom_freq_ratio']))
                totalFeatureLists[axis+'_spectral_flatness'].append( float(domFreqValues['_spectral_flatness']))
                totalFeatureLists[axis+'_spectral_entropy'].append( float(domFreqValues['_spectral_entropy']))
                
                 
        
                
                if axis == 'SVM':
                    # auto-correlation between x-y-z axes
                    crossCorXY = pearsonr(windowData, filteredData[day]['Y'][windowStart : windowStart+tDelta])
                    crossCorXZ = pearsonr(windowData, filteredData[day]['Z'][windowStart : windowStart+tDelta])
                    crossCorYZ = pearsonr(filteredData[day]['Y'][windowStart : windowStart+tDelta], filteredData[day]['Z'][windowStart : windowStart+tDelta])
                    totalFeatureLists['crossCor_XY'].append(crossCorXY[0])
                    totalFeatureLists['crossCor_XZ'].append(crossCorXZ[0])
                    totalFeatureLists['crossCor_YZ'].append(crossCorYZ[0])
                    
                    # spectral variability and approximation of smoothness and PSD-line-length (importance ref by Beck 2019, Balasubramanian 20120)
                    normPSD = psd/psd[0] # normalize PSD by first value DC-normalization (Subramaninian '12')
                    spectralVar = np.var(normPSD[lowFreq])
                    totalFeatureLists[axis+'_spectralVar'].append(spectralVar)
                    # approximation of spectral length; find_peaks finds all small peaks, 
                    # thresholds represent the distance from the peak to the neighbouring points
                    # sum of threshold-values indicates the distances the PSD-line makes to the peaks
                    ind, treshs = find_peaks(normPSD[lowFreq],height=None, threshold=0.05, distance=1) # first value returns peak-indices, second tresholds                  
                    spectralSmoothness = np.sum(treshs['left_thresholds']+treshs['right_thresholds'])
                    totalFeatureLists[axis+'_spectralSmoothness1'].append(spectralSmoothness)
                    # number of peaks found
                    spectralPeaks = len(ind)
                    totalFeatureLists[axis+'_spectrallowPeaks'].append(spectralPeaks)
                    
                    # same smoothness and peaks, with higher peak-threshold
                    ind, treshs = find_peaks(normPSD[lowFreq],height=None, threshold=0.1, distance=2) # first value returns peak-indices, second tresholds                  
                    spectralSmoothness = np.sum(treshs['left_thresholds']+treshs['right_thresholds'])
                    totalFeatureLists[axis+'_spectralSmoothness2'].append(spectralSmoothness)
                    # number of peaks found
                    spectralPeaks = len(ind)
                    totalFeatureLists[axis+'_spectralhighPeaks'].append(spectralPeaks)

        ''' All features are calculated over all axes; 
        now writing all lists with features in to a feature dataframe per session in featureDict'''
        
        # fill every column with calculated feature values
        for col in totalFeatLabels:
            features[day][col] = totalFeatureLists[col]
    
        # save features per patient
        fileName = '%s_%s%s%s_%isec_features.csv' % (pt, year,month,day, windowLen)
        csv_full_file_write = os.path.join(path,'results',pt,fileName)
        features[day].to_csv(csv_full_file_write, index=False)
    
    return features
             






In [10]:
# execute data filtering and feature extraction

filteredData = load_filterWatchData('RCS02', 2020,6,8, 2020,6,11)
features = extractFeatures(pt='RCS02', filteredData=filteredData, )



no file for RCS02_08Jun2020_userAccel.csv
no file for RCS02_09Jun2020_userAccel.csv
doch
doch
